In [2]:
import numpy as np
import h5py
import cv2
FILE_PATH = r"C:\VisualOdometry\falcon_indoor_flight_1_data.h5"

In [3]:
with h5py.File(FILE_PATH, "r") as file:
    w, h = list(file["prophesee"]["left"]["calib"]["resolution"])

In [4]:
def event_generator(file_path, cam_direction="left", batch_size=1024):
    with h5py.File(file_path, "r") as file:
        x = file["prophesee"][cam_direction]["x"]
        y = file["prophesee"][cam_direction]["y"]
        p = file["prophesee"][cam_direction]["p"]
        t = file["prophesee"][cam_direction]["t"]
        total = x.shape[0]
        for i in range(0, total, batch_size):
            yield (
                x[i : i + batch_size],
                y[i : i + batch_size],
                p[i : i + batch_size],
                t[i : i + batch_size],
            )

In [5]:
def event_frame(coords, polarities, img_shape):
    matrix = np.ones(img_shape, dtype=np.uint8) * 127
    coords = np.array(coords)
    if len(coords) > 0:
        y = coords[:, 0]
        x = coords[:, 1]
        matrix[y, x] = np.where(np.array(polarities) > 0, 255, 0)
    return matrix

In [ ]:
def odom(current_odom_map, picture):
    return current_odom_map

In [7]:
cnt = 1
batch_idx = 0
temp_x, temp_y, temp_p, temp_t = [], [], [], []

exit = False
traj = np.zeros((600, 600, 600, 3), dtype=np.uint8)
# default for left camera
for x_batch, y_batch, p_batch, t_batch in event_generator(FILE_PATH, batch_size=10000):
    batch_idx += 1
    if batch_idx % 10 != 0:
        continue

    temp_x.extend(x_batch)
    temp_y.extend(y_batch)
    temp_p.extend(np.where(p_batch > 0, 1, -1))
    temp_t.extend(t_batch)

    while len(temp_t) > 0 and temp_t[-1] > cnt * 1e5//3:
        cnt += 1
        coords = list(zip(temp_y, temp_x))
        img = event_frame(coords, temp_p, (h, w))
        
        # TODO Implement Optical flow cv2...LK...
        # and odometry from labs
        odom(traj, img)
        cv2.imshow("Event Frame", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            exit = True
            break
        temp_x.clear()
        temp_y.clear()
        temp_p.clear()
        temp_t.clear()
    if exit:
        break

cv2.destroyAllWindows()

C:\Users\mateu\AppData\Local\Temp\ipykernel_10144\2379963195.py:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, z = int(cur_t[0]) + 300, int(cur_t[2]) + 300


In [8]:
traj = np.zeros((600, 600, 3), dtype=np.uint8)
with h5py.File(FILE_PATH, "r") as file:
    rgb_dataset = file["ovc"]["rgb"]["data"]
    num_images = rgb_dataset.shape[0]
    for i in range(num_images):
        rgb = rgb_dataset[i]
        rgb = np.array(rgb, dtype=np.uint8)
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
        traj = odom(traj, gray)

        cv2.imshow("Grayscale OVC Image", gray)
        cv2.imshow("Trajectory Map", traj)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cv2.destroyAllWindows()

C:\Users\mateu\AppData\Local\Temp\ipykernel_10144\2379963195.py:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, z = int(cur_t[0]) + 300, int(cur_t[2]) + 300
